# Exercise 1
## Character-level recurrent sequence-to-sequence model
### By: Daniel Mehta

---

## Inports

In [5]:
import numpy as np
import os
from pathlib import Path

---

## Dataset Path Setup

In [6]:
# setting path to dataset
data_dir = Path("fra-eng")
data_path = data_dir/"fra.txt"

In [7]:
if not data_path.exists():
    raise FileNotFoundError(f"Dataset not found at {data_path}")

print(f"Dataset located at: {data_path}")

Dataset located at: fra-eng\fra.txt


---

## Data Exploration and Cleaning

In [8]:
# Reading the file and split into lines
with open(data_path,"r", encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

In [9]:
print(f"Total sentence pairs in file: {len(lines)}")
print("Sample lines:")
for i in range(5):
    print(lines[i])

Total sentence pairs in file: 237838
Sample lines:
Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)


In [10]:
# Separating into English and French
pairs =[line.split("\t") for line in lines]
english_sentences =[pair[0] for pair in pairs]
french_sentences =[pair[1] for pair in pairs]

In [11]:
print("\nExample pair:")
print("EN:",english_sentences[0])
print("FR:",french_sentences[0])


Example pair:
EN: Go.
FR: Va !
